Firstly get all variables we are considering appended to a raw dataframe. 
Work on the 2visit first as its the largest.

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

non_prog = pd.read_csv('Cohorts\\2visit_nonprogressor.csv')
prog = pd.read_csv('Cohorts\\2visit_progressors.csv')


# The first two columns will be patient ID and their label (progressor or non-progressor)
# The rest of the columns will be the features
# Extract IDs from the dataframes
non_prog_ids = non_prog.iloc[0, 0].strip("[]").replace("'", "").split(", ")
prog_ids = prog.iloc[0, 0].strip("[]").replace("'", "").split(", ")

# Create a new dataframe with IDs and labels
ids = non_prog_ids + prog_ids
labels = [0] * len(non_prog_ids) + [1] * len(prog_ids)

df_combined = pd.DataFrame({'ID': ids, 'Prog_ID': labels})

# Print the dataframe to verify
print(df_combined)


             ID  Prog_ID
0    NACC063570        0
1    NACC146119        0
2    NACC149720        0
3    NACC176310        0
4    NACC230335        0
..          ...      ...
849  NACC255196        1
850  NACC308030        1
851  NACC344089        1
852  NACC428778        1
853  NACC811589        1

[854 rows x 2 columns]


In [5]:
def add_columns(df, new_columns):
    for column_name, column_value in new_columns.items():
        if callable(column_value):
            df[column_name] = df.apply(column_value, axis=1)
        else:
            df[column_name] = column_value
    return df
uds = pd.read_csv('uds.csv')
sex = 'SEX'
educ = 'EDUC'
#

# Example usage:
# Define new columns to add
new_columns = {
    'New_Column_1': [1] * len(df_combined),  # Add a column with constant value
    'New_Column_2': lambda row: 1 if row['Prog_ID'] == 1 else 0  # Add a column based on a function
}

# Add new columns to the dataframe
df_combined = add_columns(df_combined, new_columns)

# Print the updated dataframe to verify
print(df_combined.head())

C:\Users\Andrew\AppData\Local\Temp\ipykernel_15424\3636059713.py:8: DtypeWarning: Columns (20,22,24,26,28,41,44,46,48,51,61,63,65,67,69,71,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,134,156,165,176,179,189,217,220,222,224,226,228,230,232,234,236,238,240,242,244,246,248,250,252,254,256,258,260,262,264,266,268,270,272,382,397,399,401,419,421,423,432,445,454,494,574,605,613,638,674,690,704,707,710,715,727,738,744,746,803,804,809,810,811,812,820,831,833,835,837,843,904,959,960,961,969,970,971,972,982,1004,1007,1010) have mixed types. Specify dtype option on import or set low_memory=False.
  uds = pd.read_csv('uds.csv')


           ID  Prog_ID  New_Column_1  New_Column_2
0  NACC063570        0             1             0
1  NACC146119        0             1             0
2  NACC149720        0             1             0
3  NACC176310        0             1             0
4  NACC230335        0             1             0
